<a href="https://colab.research.google.com/github/GailB-ca/digital-display-reader/blob/main/Digital_Reader_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (3,468 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [8]:
!pip install flask
!pip install flask-ngrok
!pip install flask-restful

     |████████████████████████████████| 61kB 3.6MB/s 


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks')

In [18]:
# import Flask and jsonify
from flask import Flask, render_template, jsonify, request
from flask_ngrok import run_with_ngrok
# import Resource, Api and reqparser
from flask_restful import Resource, Api, reqparse
import numpy as np
import lhl_model_utils as lhl_model
import lhl_build_dataframe as lhl_df
import importlib
importlib.reload(lhl_model)
importlib.reload(lhl_df)
import pandas as pd
import cv2

app = Flask(__name__, template_folder='/content/drive/MyDrive/Colab Notebooks/templates')

api = Api(app)

app.config['MAX_CONTENT_LENGTH'] = 6000 * 6000
app.config['UPLOAD_EXTENSIONS'] = ['.jpg', '.png', '.gif']

# Load the classifier model 
classifier_model = lhl_model.load_model('/content/drive/MyDrive/Colab Notebooks/digit_classifier_model1')
    
# Create a /read_digits resource for the API
class ReadDigits(Resource):
    def post(self):
        
        file = request.files['file']
        npimg = np.fromfile(file, np.uint8)
        file = cv2.imdecode(npimg, cv2.IMREAD_COLOR)

        
        
        #json_data = request.get_json()
        #df = pd.DataFrame(json_data.values(), index=json_data.keys()).transpose()

        #res = lhl.predict_digits(images_list, classifier_model)
        
        # we cannot send numpy array as a result
        #return res.tolist() 
    
    def get(self):
        return render_template('digit_classifier_index.html')

# assign endpoint to the resource
api.add_resource(ReadDigits, '/read_digits_api')        

@app.route('/read_digits', methods=['GET', 'POST'])
def read_digits():
    if request.method == 'GET':
        return render_template('digit_classifier_index.html')

    if request.method == 'POST':
        print('Its a post')
        
        file = request.files['file']
        npimg = np.fromfile(file, np.uint8)
        img_arr = cv2.imdecode(npimg, cv2.IMREAD_COLOR)

        print(img_arr.shape)
        
        images_list = np.expand_dims(img_arr, axis=0)
        images_res_list = lhl_df.process_resize_with_distortion(images_list)
        res = lhl_model.predict_digits(images_res_list, classifier_model)
        
        print(res)
        # we cannot send numpy array as a result
        #return res.tolist() 
        
        return render_template('digit_classifier_response.html')
        

#if __name__ == '__main__':
#    print("Running")
#    app.run(debug=True, host='0.0.0.0', port=5555)

run_with_ngrok(app)
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9ac6cc6b6327.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [21/Mar/2021 01:49:41] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 01:49:41] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [21/Mar/2021 01:49:45] "GET /read_digits HTTP/1.1" 200 -


Its a post
(148, 341, 3)


127.0.0.1 - - [21/Mar/2021 01:49:54] "POST /read_digits HTTP/1.1" 200 -


[2]
